In [1]:
!pip install -q -U transformers bitsandbytes accelerate

In [5]:
!git clone https://github.com/unclecode/crawl4ai.git

Cloning into 'crawl4ai'...
remote: Enumerating objects: 10463, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 10463 (delta 13), reused 17 (delta 7), pack-reused 10439 (from 1)
Receiving objects: 100% (10463/10463), 144.47 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (6734/6734), done.


In [1]:
# Entra na pasta do repositório
%cd /content/crawl4ai

# Verifica se agora você está no lugar certo (deve mostrar o caminho terminando em /crawl4ai)
!pwd

/content/crawl4ai
/content/crawl4ai


In [18]:
# Salva o histórico de merges das branches
!git log --graph --oneline --all --decorate --merges > /content/historico_merges.txt

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

In [2]:
# lê os principais arquivos com informações relacionadas às releases e as branches
changelog = ""
releases = ""
merges = ""
with open("/content/crawl4ai/CHANGELOG.md", "r") as f:
  changelog = f.read()
with open("/content/crawl4ai/.github/workflows/release.yml", "r") as f:
  releases = f.read()
with open("/content/historico_merges.txt", "r") as f:
  merges = f.read()

In [3]:
# criação do prompt
system_msg = f"""Você é um Especialista em Engenharia de Software com foco em Gerenciamento de Configuração. Sua tarefa é analisar o repositório "Crawl4AI" para identificar sua Estratégia de Release e seu Fluxo de Trabalho (Branching Model).

PRINCIPAIS ESTRATÉGIAS DE WORKFLOW PARA PROCURAR:
- Gitflow: Presença de branches persistentes 'main' E 'develop'. Merges de features vão para 'develop'. Existem branches de 'release/v*'.
- GitHub Flow: Apenas a branch 'main' é persistente. Features branches são curtas e mergeadas direto na 'main'.
- Trunk-based: Commits frequentes e diretos na 'main' (ou via PRs curtíssimos). O histórico é quase uma linha única e reta.

PRINCIPAIS ESTRATÉGIAS DE RELEASE PARA PROCURAR:
- Rapid Release: Alta frequência de versões (tags v* constantes) baseadas em novas funcionalidades prontas.
- Release Train: Versões em datas fixas e previsíveis.
- LTS + Current: Coexistência de versões estáveis de longo suporte com versões de ponta.

VOCÊ RECEBERÁ DADOS DOS SEGUINTES ARQUIVOS:
- CHANGELOG.md: Registro histórico de versões e datas.
- release.yml: Configuração de automação do GitHub Actions.
- Histórico de Merges: O log visual (graph) da árvore de commits.

METODOLOGIA DE ANÁLISE:
1. Examine o 'release.yml': Tente identificar o que dispara o deploy.
2. Examine o 'CHANGELOG.md': Tente identificar o intervalo médio entre versões.
3. Examine o 'Histórico de Merges': Procure pela existência da branch 'develop' e branches com prefixo 'release/v*', 'feature/*' ou 'fix/* e verificar a persistência dessas branches'.

CONCLUSÃO:
Apresente sua conclusão baseada na predominância das evidências. Se houver elementos de mais de um modelo (ex: um fluxo Gitflow simplificado com ritmo de Rapid Release), descreva essa governança híbrida."""
user_msg = f"""
Aqui estão os arquivos a serem analisados:
CHANGELOG.md:
{"\n".join(changelog.splitlines()[-262:])}

release.yml:
{releases}

Histórico de Merges:
{merges}
"""

In [4]:
# modelo
model_id = "microsoft/Phi-3.5-mini-instruct"

# Configuração de Quantização usando bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Carregar Tokenizer e Modelo
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=False)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=False,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="sdpa"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg},
    ]

inputs = tokenizer.apply_chat_template(
  messages,
  add_generation_prompt=True,
  tokenize=True,
  return_dict=True,
  return_tensors="pt",
).to(model.device)

#. Gerar a Resposta
outputs = model.generate(**inputs, max_new_tokens=1500, use_cache=True, do_sample=True, temperature=0.2, top_p=0.9, repetition_penalty=1.1)
with open("/content/resposta.md", "w") as f:
  f.write(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True))